In [13]:
import numpy as np

def euclidean_dist(a, b):
    return np.linalg.norm(a - b)

In [15]:
def dist_matrix(X):
    m = X.shape[0]
    D = np.zeros((m, m))
    for i in range(m):
        for j in range(i, m):
            D[i][j] = D[j][i] = euclidean_dist(X[i], X[j])
    return D

In [17]:
class DSU:
    def __init__(self, size):
        self.parent = list(range(size))

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        xr = self.find(x)
        yr = self.find(y)
        if xr != yr:
            self.parent[yr] = xr

In [19]:
def dbscan(X, epsilon=0.5, min_samples=3):
    m = X.shape[0]
    
    dsu = DSU(m)
    D = dist_matrix(X)

    core_points = []
    for i in range(m):
        neighbors = np.where(D[i] <= epsilon)[0]
        if len(neighbors) >= min_samples:
            core_points.append(i)
            for j in neighbors:
                dsu.union(i, j)

    labels = [-1] * m  # -1 means noise 
    cluster_map = {}
    cluster_id = 0

    for i in core_points:
        root = dsu.find(i)
        if root not in cluster_map:
            cluster_map[root] = cluster_id
            cluster_id += 1
        labels[i] = cluster_map[root]

    for i in range(m):
        if labels[i] == -1:
            for j in core_points:
                if D[i][j] <= epsilon:
                    labels[i] = labels[j]
                    break

    return labels

In [21]:
X = np.array([[1, 2], [1, 3], [1, 4],
              [10, 10], [10, 11], [25, 25]])

labels = dbscan(X, epsilon=1.5, min_samples=2)
print("Cluster Labels:", labels)

Cluster Labels: [0, 0, 0, 1, 1, -1]
